### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
from neuropy.analyses.placefields import Pf1D
import subjects

### Placefields and population vector correlation MAZE vs REMAZE

In [ ]:
sessions = subjects.remaze_sess()[1:]  # RatSDay2NSD does not have remaze position info
kw = dict(frate_thresh=0, grid_bin=5)

tc_all = []
for s, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")

    # ---- Run direction sepearted-------
    tc_maze, tc_remaze = [], []
    for direction in ["up", "down"]:
        maze_run = sess.maze_run[direction]
        pfmaze = Pf1D(neurons, position=sess.maze, epochs=maze_run, **kw)
        tc_maze.append(pfmaze.resample(50).tuning_curves)

        remaze_run = sess.remaze_run[direction]
        pfremaze = Pf1D(neurons, position=sess.remaze, epochs=remaze_run, **kw)
        tc_remaze.append(pfremaze.resample(50).tuning_curves)

    tc_comb = np.hstack((np.vstack(tc_maze), np.vstack(tc_remaze)))

    # ---- Overall----
    # pfmaze = Pf1D(neurons, position=sess.maze, **kw)
    # tc_maze = pfmaze.resample(50).tuning_curves

    # pfremaze = Pf1D(neurons, position=sess.remaze, **kw)
    # tc_remaze = pfremaze.resample(50).tuning_curves

    # tc_comb = np.hstack((tc_maze, tc_remaze))

    tc_comb_df = pd.DataFrame(tc_comb)
    tc_comb_df["session"] = s
    tc_comb_df["grp"] = sess.tag
    tc_all.append(tc_comb_df)

tc_all = pd.concat(tc_all, ignore_index=True)

In [ ]:
from neuropy.utils.mathutil import min_max_scaler
from scipy.ndimage import gaussian_filter1d, gaussian_filter

fig = plotting.Fig(11, 8)

for g, grp in enumerate(["NSD", "SD"]):
    tc_grp = tc_all[tc_all.grp == grp].iloc[:, :100].to_numpy()
    # tc_grp = gaussian_filter1d(tc_grp, sigma=1.5, axis=1)
    tc_new = np.zeros_like(tc_grp)
    tc_new[:, :50] = min_max_scaler(tc_grp[:, :50], axis=1)
    tc_new[:, 50:] = min_max_scaler(tc_grp[:, 50:], axis=1)
    # tc_new =  min_max_scaler(tc_grp,axis=-1)
    sort_ind = np.argsort(np.argmax(tc_new[:, :50], axis=1))
    tc_new = gaussian_filter1d(tc_new, sigma=1.5, axis=1)
    print(tc_new.shape)

    ax = fig.subplot(fig.gs[:2, g + 1])
    pf_im = ax.pcolormesh(tc_new[sort_ind, :], cmap="binary", rasterized=True)
    ax.axvline(50, color="k")
    ax.set_xticks([0, 50, 51, 100], [0, 1, 0, 1])
    ax.set_ylabel("Neurons")
    ax.set_xlabel("Position")
    pf_cb = plt.colorbar(pf_im, anchor=(1, 1), shrink=0.3, ticks=[0, 0.9])

    ax = fig.subplot(fig.gs[2, g + 1])
    corr = gaussian_filter(np.corrcoef(tc_grp.T), 1)
    np.fill_diagonal(corr, 0)
    corr_im = ax.pcolormesh(corr, cmap="bwr", vmin=0.1, vmax=0.8, rasterized=True)
    ax.axvline(50, color="k")
    ax.axhline(50, color="k")
    ax.axis("off")
    corr_cb = plt.colorbar(corr_im, shrink=0.5, anchor=(1, 1), ticks=[0.1, 0.5, 0.8])


# fig.savefig(subjects.figpath_sd / "maze_remaze_pf_run_direction")